### Gathering Data from NamUs

NamUs is the National Missing and Unidentified Persons System, which is financed by the United States Department of Justice. NamUs does not provide an API, however it does provide a searchable interface.

In this notebook, we will use Selenium with beautifulSoup to retrieve and save data from NamUs as CSV.

In [1]:
# Import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# initialize global driver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [3]:
# constants
LOCATION = ['Maine']
COLUMNS = [
    'Case Number',
    'DLC',
    'Last Name',
    'First Name',
    'Missing Age',
    'City',
    'County',
    'State',
    'Sex',
    'Race',
    'Date Modified'
]

In [4]:
# find state filter
driver.get("https://www.namus.gov/MissingPersons/Search")
section_on_circumstances = driver.find_element_by_id('Circumstances')
labels_in_section = section_on_circumstances.find_elements_by_tag_name('label')

state_input_box = None

for label in labels_in_section:
    if (label.text == "State"):
        state_input_box = label.find_element_by_tag_name('input')
        # add state filter
        for state in LOCATION:
            state_input_box.send_keys(state)
            state_input_box.send_keys(Keys.ENTER)


In [5]:
# navigate to list view
buttons_action = driver.find_elements_by_class_name('button-box')[0].find_elements_by_tag_name('input')
buttons_action[1].click()

In [6]:
# show 100 results at a time
time.sleep(1.5)
dropdown_selection_results = driver.find_element_by_xpath("//label/span[contains(text(),'Results')]/following-sibling::select")
Select(dropdown_selection_results).select_by_value('100')
time.sleep(1.5)

In [7]:
# get row results and store in dataframes
df = pd.DataFrame(columns=COLUMNS)
soup = BeautifulSoup(driver.page_source, 'lxml')
rows = soup.find('div', class_='ui-grid-canvas').contents

for row in rows:
    if row != ' ':
        cells = row.find_all('div', class_='ui-grid-cell-contents')
        cells_text = map(lambda cell: cell.text.strip(), cells)
        new_df = pd.DataFrame([list(cells_text)], columns=COLUMNS)
        df = df.append(new_df, ignore_index=True)


C:\Users\rapid\AppData\Local\Temp\ipykernel_12060\1981749166.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)
C:\Users\rapid\AppData\Local\Temp\ipykernel_12060\1981749166.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)
C:\Users\rapid\AppData\Local\Temp\ipykernel_12060\1981749166.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)
C:\Users\rapid\AppData\Local\Temp\ipykernel_12060\1981749166.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)
C:\Users\rapid\AppDa

In [8]:
# navigate to grid view
driver.find_element_by_xpath("//i[@class=\"icon-grid-six\"]").click()
driver.quit() 


In [9]:
df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race,Date Modified
0,MP92565,06/09/2022,Conaway,James,66 Years,Big W Township,Somerset,ME,Male,White / Caucasian,06/24/2022
1,MP93172,06/06/2022,Lacher,Graham,37 Years,Bangor,Penobscot,ME,Male,White / Caucasian,07/11/2022
2,MP91650,04/13/2022,Carver,Randy,66 Years,Holden,Penobscot,ME,Male,White / Caucasian,07/12/2022
3,MP90300,02/18/2022,Paradee,Jared,43 Years,Rockport,Knox,ME,Male,White / Caucasian,04/05/2022
4,MP89506,02/11/2022,Lang,Nicholas,41 Years,Kittery,York,ME,Male,White / Caucasian,03/08/2022
...,...,...,...,...,...,...,...,...,...,...,...
95,MP19763,08/24/1986,Meyer,Kaye,80 Years,Monson,Piscataquis,ME,Female,White / Caucasian,06/24/2022
96,MP35491,08/11/1986,Simpson,Harold,28 Years,Livermore,Androscoggin,ME,Male,White / Caucasian,06/24/2022
97,MP35636,06/06/1986,Thompson,Stephen,38 Years,Oakland,Kennebec,ME,Male,White / Caucasian,12/03/2021
98,MP23215,06/02/1986,Letarte,Philip,64 Years,Woodland,Aroostook,ME,Male,White / Caucasian,12/03/2021
